# Manipulating Data in DataFrames HW


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [66]:
import pandas as pd
import numpy as np
import datetime as dt
import re

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *#avg, count, expr
from pyspark.sql.types import *

In [3]:
# initialize
sc = pyspark.SparkContext()
spark = SparkSession(sc)
spark.sparkContext.appName = 'manipulateAss'
# show the number of cores
print('%d cores'%spark._jsc.sc().getExecutorMemoryStatus().keySet().size())
spark

1 cores


## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [15]:
# load the data
fil = '../../data/Rep_vs_Dem_tweets.csv'
schem = StructType([StructField('Party', StringType()),
                    StructField('Handle', StringType()),
                    StructField('Tweet', StringType())])
tweets = spark.read.format('csv').options(header=True).schema(schem).load(fil).repartition('Party')

## About this dataframe

Extracted tweets from all of the representatives (latest 200 as of May 17th 2018)

**Source:** https://www.kaggle.com/kapastor/democratvsrepublicantweets#ExtractedTweets.csv

Use either .show() or .toPandas() check out the first view rows of the dataframe to get an idea of what we are working with.

In [9]:
# talk
print('%d records'%tweets.count())
display(tweets.show(truncate=False))

92489 records
+------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+---------------------------------------------+
|Party                                                                                                                   |Handle                                                                                |Tweet                                        |
+------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+---------------------------------------------+
|investigat…"                                                                                                            |null                                                                                  |null     

None

**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [ ]:
# no need

**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [ ]:
# no need, as I set the schema

## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [18]:
tweets.select('*', col('Tweet').like('%@LatinoLeader%').alias('seld')).where(col('seld')==True).show(5, truncate=False)

+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+----+
|Party   |Handle       |Tweet                                                                                                                                       |seld|
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+----+
|Democrat|RepDarrenSoto|RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…|true|
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+----+



## 2. Replace any value other than 'Democrate' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrate' or 'Republican' in the Part column. We are assuming that this is dirty data that needs to be cleaned up.

In [23]:
tweets = tweets.select(when(col('Party').isin(['Democrat', 'Republican']), col('Party')).otherwise('Other').alias('Party'), 'Handle', 'Tweet')

tweets.groupBy(col('Party')).count().show()

+----------+-----+
|     Party|count|
+----------+-----+
|Republican|44392|
|  Democrat|42068|
|     Other| 6029|
+----------+-----+



## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

In [41]:
tweets = tweets.select('Party', 'Handle', trim(substring_index(col('Tweet'), 'https', 1)).alias('Tweet'))\
    .where(col('Tweet').isNotNull() & col('Handle').isNotNull())
tweets.show(truncate=False)

+----------+----------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party     |Handle                                                          |Tweet                                                                                                                                       |
+----------+----------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Other     | Florida                                                        |1963. More photos:                                                                                                                          |
|Other     | I will join @alisonstewart on @WNYCMidday to discuss my novel  |“Big Guns.”                                    

## 4. Remove any leading or trailing white space in the tweet column

In [ ]:
# already did that

## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [42]:
tweets.withColumnRenamed('Party', 'Dem_Rep_Other').show(truncate=False)

+-------------+----------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Dem_Rep_Other|Handle                                                          |Tweet                                                                                                                                       |
+-------------+----------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Other        | Florida                                                        |1963. More photos:                                                                                                                          |
|Other        | I will join @alisonstewart on @WNYCMidday to discuss my novel  |“Big Guns.”                     

## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [43]:
tweets.select(concat_ws('-', col('Party'), col('Handle')).alias('Who'), 'Tweet').show(truncate=False)

+----------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Who                                                                   |Tweet                                                                                                                                       |
+----------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Other- Florida                                                        |1963. More photos:                                                                                                                          |
|Other- I will join @alisonstewart on @WNYCMidday to discuss my novel  |“Big Guns.”                                                             

## Challenge Question

Let's image that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

In [77]:
def refa(x):
    res = list(re.findall('\#[A-Za-z0-9]*', x))
    return res
refaUDF = udf(lambda x: refa(x), ArrayType(StringType(), True))

In [79]:
# can't get this to match all of the patterns in the tweet :-(
thisPatt = '(\#[A-Za-z0-9]*)'
tweets.select((length(col('Tweet')) - length(array_join(split('Tweet', '#'), ''))).alias('mult'),
              regexp_extract(col('Tweet'), thisPatt, 0).alias('Hashtag'), 'Tweet')\
    .orderBy(col('mult').desc()).show(truncate=False)

+----+-----------------------+------------------------------------------------------------------------------------------------------------------------------------------------+
|mult|Hashtag                |Tweet                                                                                                                                           |
+----+-----------------------+------------------------------------------------------------------------------------------------------------------------------------------------+
|7   |#GoldenPlow            |In cause you missed it yesterday! #GoldenPlow #Farmer #Rancher #TaxReform #FarmBill #Biodiesel #StopTheHIT…                                     |
|7   |#Attention             |RT @FAANews: #Attention #drone #pilots: Do not fly your #drone near the #KilaueaVolcano in #Hawaii. The #FAA has flight restrictions in pla…    |
|7   |#Rally                 |Thank you @HRCLasVegas for continuing to #Rally #Ignite #Support &amp; #Empower #HRCGala #

# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients first and last names, date of birth, and the dates of their first 3 visits. 

In [3]:
from pyspark.sql.types import *

md_office = [('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2') \
            ,('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2') \
            ,('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2') \
            ,('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2') \
            ,('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2') \
            ,('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2') \
            ,('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2') ]

df = spark.createDataFrame(md_office,['first_name','last_name','dob','visit1','visit2','visit3']) # schema=final_struc

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)

None


Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

## 8. Can you calculate the age of each patient?

## 9. Can you extract the month from the first visit column and call it "Month"?

## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [3]:
from pyspark.sql.functions import *

values = [(5,'Epic. This is the best movie I have EVER seen'), \
          (4,'Pretty good, but I would have liked to seen better special effects'), \
          (3,'So so. Casting could have been improved'), \
          (5,'The most EPIC movie of the year! Casting was awesome. Special effects were so intense.'), \
          (4,'Solid but I would have liked to see more of the love story'), \
          (5,'THE BOMB!!!!!!!')]
reviews = spark.createDataFrame(values,['rating', 'review_txt'])

reviews.show(6,False)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|4     |Pretty good, but I would have liked to seen better special effects                    |
|3     |So so. Casting could have been improved                                               |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
|4     |Solid but I would have liked to see more of the love story                            |
|5     |THE BOMB!!!!!!!                                                                       |
+------+--------------------------------------------------------------------------------------+



## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the rview_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

### That's it! Great Job!